<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Advanced MetPy: Isentropic Analysis</h1>
    <h3>AMS 2022 Short Course: MetPy for Quantitative Analysis of Meteorological Data</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

### Tasks
1. <a href="#download">Access GFS output from a THREDDS Data Server (TDS)</a>
1. <a href="#interpolation">Isentropic interpolation</a>
1. <a href="#ascent">Calculate isentropic ascent ($\omega$)</a>
1. <a href="#mixingratio">Calculate mixing ratio</a>


### Background
Up until this point, we have been examining data on constant pressure (isobaric) surfaces, which aligns with how meteorological data are typically written and served. However, we know there are several benefits for examining data on constant potential temperature (isentropic) surfaces including diagnosing vertical motion through advection of pressure, locating airmass boundaries in a conveyor belts framework, depicting the dynamic tropopause and tropopause folds, among <a href="https://www.meted.ucar.edu/labs/synoptic/isentropic_analysis/print.php" target="blank">others</a>. 

<center><img src="https://elearning.unidata.ucar.edu/metpy/AMS2022/thetaxsect2.png" width="800px"/><br>
<i>Vertical cross-section of potential temperature</i><br>
<i>An isentropic chart uses an isentrope as its base level</i></center><br>


In this lesson, we will demonstrate how to use built-in MetPy functions to easily interpolate variables on isobaric surfaces to isentropic surfaces. 

<a name="download"></a>
## Step 1: Access GFS Output
We will perform the isentropic interpolation in this lesson on the latest run of the GFS available on [Unidata's THREDDS data server](https://thredds.ucar.edu/). First, we access the catalog for the half-degree GFS output, and look for the dataset called the "[Best GFS Half Degree Forecast Time Series](https://thredds.ucar.edu/thredds/catalog/grib/NCEP/GFS/Global_0p5deg/catalog.html?dataset=grib/NCEP/GFS/Global_0p5deg/Best)". This dataset combines multiple sets of model runs to yield a time series of output with the shortest forecast offset.

In [ ]:
from siphon.catalog import TDSCatalog

cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog/grib/'
                 'NCEP/GFS/Global_0p5deg/catalog.xml')
best = cat.datasets['Best GFS Half Degree Forecast Time Series']

Next, we use the [NetCDF Subset Service (NCSS)](https://www.unidata.ucar.edu/software/tds/current/reference/NetcdfSubsetServiceReference.html) to make requests from the GRIB collection and get results in netCDF format.

In [ ]:
subset_access = best.subset()
query = subset_access.query()

GFS output contains [many variables](https://thredds.ucar.edu/thredds/ncss/grib/NCEP/GFS/Global_0p25deg/Best/dataset.html). In fact, too many to search for all the variables we need for this interpolation. Instead of printing `subset_access.variables` and listing all 150+ variable names, we can ask Python to only display variables that end with "isobaric". This suffix is how the TDS denotes GRIB fields that are specified on isobaric levels.

In [ ]:
sorted(v for v in subset_access.variables if v.endswith('isobaric'))

#### Query the TDS

Now we put together the query to send to the TDS. Recall that we can query for a bounding box and time in addition to the variables we need. 

In this example, we'll query for data across the contenental US and for the time step closest to now. We also request **temperature**, **height**, **winds**, and **relative humidity**. By asking for netCDF4 data, the result is compressed, so the download is smaller.

In [ ]:
from datetime import datetime
dt = datetime.utcnow()
query.time(dt)
query.variables('Temperature_isobaric', 'Geopotential_height_isobaric',
                'u-component_of_wind_isobaric', 'v-component_of_wind_isobaric',
                'Relative_humidity_isobaric')
query.lonlat_box(west=-130, east=-50, south=10, north=60)
query.accept('netcdf4')

Now all that's left is to actually make the request for data.

In [ ]:
nc = subset_access.get_data(query)

#### Open and examine data

Now we can open and review the returned netCDF data using XArray. Notice the dimensions each of the variables have. 

In [ ]:
from xarray.backends import NetCDF4DataStore
import xarray as xr
ds = xr.open_dataset(NetCDF4DataStore(nc))
ds

Since we are working with only one time, we can use `squeeze()` to remove time as a dimension from the dataset. In the same step, we can also parse out the metadata for the isobaric temperature and get the projection information with `parse_cf()`.

In [ ]:
import metpy
ds = ds.metpy.parse_cf().squeeze()
ds

<a name="interpolation"></a>
## Step 2: Isentropic Interpolation

We can now use MetPy to interpolate the data we've just accessed to an isentropic surface and visualize it on a map. To transform our variables from the pressure vertical coordinate to a constant potential temperature (isentropic) vertical coordinate, we can use the <a href="https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.isentropic_interpolation.html" target="blank">`isentropic_interpolation`</a> function. Since we have several variables to interpolate, we can also use the <a href="https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.isentropic_interpolation_as_dataset.html" target="blank">`isentropic_interpolation_as_dataset`</a> function to organize all outputs into an xarray Dataset. 

First, we import the required packages for this analysis.

In [ ]:
import metpy.calc as mpcalc
from metpy.units import units
import numpy as np

Next, we perform the isentropic interpolation. For this, we will use [`isentropic_interpolation_as_dataset`](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.isentropic_interpolation_as_dataset.html). At a minimum, this must be given one or more isentropic levels and the 3-D temperature field. It then returns the 3D array of pressure values (2D slices for each isentropic level). This relies on getting the pressure values from the passed in temperature. 

There is also a version of this function, [`isentropic_interpolation`](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.isentropic_interpolation.html), that returns individual arrays rather than an xarray dataset.

You can also pass additional fields which will be interpolated to these levels as well. Below, we interpolate the winds (and pressure) to the 320 K isentropic level:

In [ ]:
isen_level = np.array([320]) * units.kelvin
isen_ds = mpcalc.isentropic_interpolation_as_dataset(isen_level, ds.Temperature_isobaric,
                                                     ds['u-component_of_wind_isobaric'],
                                                     ds['v-component_of_wind_isobaric'])

# Squeeze the single isentropic level to return a 2D array for plotting
isen_ds = isen_ds.metpy.parse_cf().squeeze()
isen_ds

#### Plot results

Let's now preview our data on the 320 K isentropic surface. 

In [ ]:
# Pull out pressure and wind (u and v) arrays from the dataset
isen_press = isen_ds.pressure
isen_u = isen_ds['u-component_of_wind_isobaric']
isen_v = isen_ds['v-component_of_wind_isobaric']

# Pull the projection for the plot
data_proj = isen_press.metpy.cartopy_crs

# Add lat/lon to pressure array
lon = isen_press.metpy.longitude
lat = isen_press.metpy.latitude

isen_press

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Create a plot and basic map projection
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.LambertConformal(central_longitude=-100))

# Contour the pressure values for the isentropic level. We keep the handle
# for the contour so that we can have matplotlib label the contours
levels = np.arange(300, 1000, 25)
cntr = ax.contour(lon, lat, isen_press, transform=data_proj,
                  colors='black', levels=levels)
cntr.clabel(fmt='%d')

# Set up slices to subset the wind barbs--the slices below are the same as `::7` and `::5`
# We put these here so that it's easy to change and keep all of the ones below matched
# up.
lon_slice = slice(None, None, 7)
lat_slice = slice(None, None, 5)
ax.barbs(lon[lon_slice], lat[lat_slice],
         isen_u[lat_slice, lon_slice].metpy.convert_units('knots').to_numpy(),
         isen_v[lat_slice, lon_slice].metpy.convert_units('knots').to_numpy(),
         transform=data_proj, zorder=2)

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linewidth=2)
ax.add_feature(cfeature.STATES, linestyle=':')

ax.set_extent((-120, -70, 25, 55), crs=data_proj)

<a name="ascent"></a>
## Step 3: Calculate Isentropic Ascent ($\omega$)

Air flow across isobars on an isentropic surface represents vertical motion. Isentropic ascent. $\omega$ is represented by:

$$\omega = \left(\frac{\partial P}{\partial t}\right)_\theta + \vec{V} \cdot \nabla_\theta P + \frac{\partial P}{\partial \theta}\frac{d\theta}{dt}$$

However, in practice we often represent $\omega$ just by the second term, the negated advection of Pressure across isentropic surfaces. We use the frozen wave approximation to remove the first term on the right hand side, and we assume that the phase speed of the system is much smaller than windspeed on the isentropic surface to eliminate the third term on the right hand side. In this example, we will calculate $\omega$ as just the second term, rewritten as:

$$-(-\vec{V} \cdot \nabla_\theta P)$$

Since calculating this involves taking derivatives, first let's smooth the input fields using a gaussian_filter.

In [ ]:
isen_press = mpcalc.smooth_gaussian(isen_press, 9)
isen_u = mpcalc.smooth_gaussian(isen_u, 9)
isen_v = mpcalc.smooth_gaussian(isen_v, 9)

<div class="alert alert-success">
    <b>EXERCISE</b>: Calculate $\omega$

With the above arrays and assumptions previously made:
<ol>
    <li>Calculate $\omega$ as a variable called <code>lift</code></li>
    <li>Check the units of the output array to ensure that the result is a velocity</li>
</ol>
   
</div>

In [ ]:
# YOUR CODE HERE

### Plot

With the `lift` variable calculated, plot it on an isentropic surface along with winds and pressure contours

In [ ]:
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.LambertConformal(central_longitude=-100))
ax.add_feature(cfeature.COASTLINE)

levels = np.arange(300, 1000, 25)
cntr = ax.contour(lon, lat, isen_press, transform=data_proj, colors='black', levels=levels)
cntr.clabel(fmt='%d')


lon_slice = slice(None, None, 5)
lat_slice = slice(None, None, 5)
ax.barbs(lon[lon_slice], lat[lat_slice],
         isen_u[lon_slice, lat_slice].metpy.convert_units('knots').to_numpy(),
         isen_v[lon_slice, lat_slice].metpy.convert_units('knots').to_numpy(),
         transform=data_proj, zorder=2)


levels = np.arange(-8, 9)
# Convert the units to microbar/s to better visualize on the plot
cs = ax.contourf(lon, lat, lift.metpy.convert_units('microbar/s').to_numpy(), levels=levels, cmap='RdBu',
                 transform=data_proj, extend='both')
plt.colorbar(cs)


ax.set_extent((-120, -70, 25, 55), crs=data_proj)

<a name="mixingratio"></a>
## Step 4: Calculate mixing ratio

Isentropic charts are often used for visualizing moisture transport, such as in warm conveyor belt systems. We can check for the presence of moisture transport by adding mixing ratio to our isentropic charts. 

<div class="alert alert-success">
    <b>EXERCISE</b>: Calculate mixing ratio
    
In this exercise you will calculate mixing ratio and visualize it on a plot similar to the ones above. 
    
This process includes several tasks:
    
 <ol>
     <li>Calculate the mixing ratio using an appropriate function from mpcalc. See: <a href="https://unidata.github.io/MetPy/latest/api/index.html" target="blank">MetPy Documentation</a></li>
     <li>Interpolate mixing ratio to an isentropic surface using the same method as previous. You may continue to use the 320 K level or another level of your choice.</li>
     <li>Use contour() or contourf() as used in previous examples to display mixing ratio alongside winds and pressure on a map.</li>
</ol>
    
Hints:
    
<ul>
    <li>Recall the variables we pulled from the TDS in the <code>ds</code> DataSet previously.</li>
    <li>Calculate mixing ratio with isobaric levels first, then interpolate to an isentropic level as in the <a href="#interpolation">Isentropic Interpolation</a> section above. </li>
    <li>If you need a pressure array, pull the vertical coordinate from one of the isobaric variables in the <code>ds</code> DataSet, <br>e.g. <code>pressure = ds.Temperature_isobaric.metpy.vertical</code></li>
    <li>If using <code>isentropic_interpolation_as_dataset</code> for the mixing ratio interpolation, be sure to add a <code>name</code> property to the DataArray first<br>e.g. <code>mixing.name = 'mixing_ratio'</code>
    <li>Remember to <code>squeeze</code> any arrays to plot to just two dimensions.</li>
    <li>If your final plot does not include much area of >1 mixing ratio, try interpolating to another isentropic level.</li>
    </ul>
</div>

In [ ]:
# YOUR CODE HERE (Calculations)

In [ ]:
# YOUR CODE HERE (Plotting)
# Previous plotting code is copied here for you to start

# Create figure
fig = plt.figure(figsize=(14, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.LambertConformal(central_longitude=-100))

# Pressure contours
levels = np.arange(200, 1000, 25)
cntr = ax.contour(lon, lat, isen_press, transform=data_proj,
                  colors='black', levels=levels)
cntr.clabel(fmt='%d')

# Wind barbs
lon_slice = slice(None, None, 8)
lat_slice = slice(None, None, 8)
ax.barbs(lon[lon_slice], lat[lat_slice],
         isen_u[lat_slice, lon_slice].metpy.convert_units('knots').to_numpy(),
         isen_v[lat_slice, lon_slice].metpy.convert_units('knots').to_numpy(),
         transform=data_proj, zorder=2)



# Mixing ratio
# 
# YOUR CODE: Contour/Contourf the mixing ratio values
# 
# 


# Geographic context
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linewidth=2)
ax.add_feature(cfeature.STATES, linestyle=':')

# Set axes extent
ax.set_extent((-120, -70, 25, 55), crs=data_proj)